### ENSEMBLE

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import xgboost
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, recall_score, roc_curve
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

In [2]:
#  Pruebo Tuneo de hiperparámetros con los datos en bruto del principio (parecen mejores que lo que he sacado)

In [3]:
df=pd.read_csv('./training set values.csv', sep=',')

In [4]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [5]:
df_target = pd.read_csv('training set labels.csv')
df = pd.merge(df, df_target)
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [6]:
variables_categoricas = [
    'funder',
    'installer',
    'wpt_name',
    'basin',
    'subvillage',
    'region',
    'lga',
    'ward',
    'public_meeting',
    'recorded_by',
    'scheme_management',
    'scheme_name',
    'permit',
    'extraction_type',
    'extraction_type_group',
    'extraction_type_class',
    'management',
    'management_group',
    'payment',
    'payment_type',
    'water_quality',
    'quality_group',
    'quantity',
    'quantity_group',
    'source',
    'source_type',
    'source_class',
    'waterpoint_type',
    'waterpoint_type_group'
]

In [7]:
# Copio y pego gran parte del codigo de V0 para el label encoder
df3 = df
le = LabelEncoder()
for variable in variables_categoricas:
    df3[variable] = le.fit_transform(df3[variable])
pd.set_option('display.max_columns', None)

In [8]:
df3


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,1368,1390,1518,34.938093,-9.856322,37398,0,1,11807,3,11,5,51,1426,109,1,0,6,2244,0,1999,3,1,0,7,4,2,0,6,2,1,1,8,6,0,1,1,functional
1,8776,0.0,2013-03-06,469,1399,545,34.698766,-2.147466,37194,0,4,15838,9,20,2,103,1576,280,2,0,1,2695,1,2010,3,1,0,11,4,0,2,6,2,2,2,5,3,1,1,1,functional
2,34310,25.0,2013-02-25,825,686,2048,37.460664,-3.821329,14572,0,5,9074,8,21,4,108,1624,250,1,0,6,2120,1,2009,3,1,0,7,4,4,5,6,2,1,1,0,1,1,2,1,functional
3,67743,0.0,2013-01-28,1740,263,1852,38.486161,-11.155298,37284,0,7,8982,12,90,63,87,1571,58,1,0,6,2695,1,1986,14,10,5,7,4,0,2,6,2,0,0,3,0,0,2,1,non functional
4,19728,0.0,2011-07-13,20,0,119,31.130847,-1.825359,35528,0,4,7698,4,18,1,26,1687,0,1,0,11,2695,1,0,3,1,0,1,1,0,2,6,2,3,3,5,3,1,1,1,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,436,1210,201,37.169807,-3.253847,513,0,5,5681,6,3,5,16,1090,125,1,0,9,1068,1,1999,3,1,0,9,4,4,5,6,2,1,1,8,6,0,1,1,functional
59396,27263,4700.0,2011-05-07,177,1212,265,35.249991,-9.070629,24074,0,6,2980,3,11,4,91,353,56,1,0,6,479,1,1996,3,1,0,7,4,2,0,6,2,1,1,6,4,1,1,1,functional
59397,37057,0.0,2011-04-11,1896,0,2145,34.017087,-8.750434,27926,0,6,8784,10,12,7,59,177,0,1,0,6,2695,0,0,15,11,1,7,4,3,1,1,1,1,1,3,0,0,4,3,functional
59398,31282,0.0,2011-03-08,884,0,1213,35.861315,-6.378573,29693,0,6,14012,2,1,4,11,1449,0,1,0,6,2695,1,0,8,5,1,7,4,0,2,6,2,2,2,7,5,0,4,3,functional


In [9]:
del df3['date_recorded'] # para que no de problemas el datatime
df3

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,1368,1390,1518,34.938093,-9.856322,37398,0,1,11807,3,11,5,51,1426,109,1,0,6,2244,0,1999,3,1,0,7,4,2,0,6,2,1,1,8,6,0,1,1,functional
1,8776,0.0,469,1399,545,34.698766,-2.147466,37194,0,4,15838,9,20,2,103,1576,280,2,0,1,2695,1,2010,3,1,0,11,4,0,2,6,2,2,2,5,3,1,1,1,functional
2,34310,25.0,825,686,2048,37.460664,-3.821329,14572,0,5,9074,8,21,4,108,1624,250,1,0,6,2120,1,2009,3,1,0,7,4,4,5,6,2,1,1,0,1,1,2,1,functional
3,67743,0.0,1740,263,1852,38.486161,-11.155298,37284,0,7,8982,12,90,63,87,1571,58,1,0,6,2695,1,1986,14,10,5,7,4,0,2,6,2,0,0,3,0,0,2,1,non functional
4,19728,0.0,20,0,119,31.130847,-1.825359,35528,0,4,7698,4,18,1,26,1687,0,1,0,11,2695,1,0,3,1,0,1,1,0,2,6,2,3,3,5,3,1,1,1,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,436,1210,201,37.169807,-3.253847,513,0,5,5681,6,3,5,16,1090,125,1,0,9,1068,1,1999,3,1,0,9,4,4,5,6,2,1,1,8,6,0,1,1,functional
59396,27263,4700.0,177,1212,265,35.249991,-9.070629,24074,0,6,2980,3,11,4,91,353,56,1,0,6,479,1,1996,3,1,0,7,4,2,0,6,2,1,1,6,4,1,1,1,functional
59397,37057,0.0,1896,0,2145,34.017087,-8.750434,27926,0,6,8784,10,12,7,59,177,0,1,0,6,2695,0,0,15,11,1,7,4,3,1,1,1,1,1,3,0,0,4,3,functional
59398,31282,0.0,884,0,1213,35.861315,-6.378573,29693,0,6,14012,2,1,4,11,1449,0,1,0,6,2695,1,0,8,5,1,7,4,0,2,6,2,2,2,7,5,0,4,3,functional


In [10]:
#df_target['status_group'].replace([0, 1, 2], ['non functional', 'functional needs repair', 'functional'], inplace=True)
df3['status_group'].replace(['non functional', 'functional needs repair', 'functional'], [0, 1, 2], inplace=True)
df3

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,1368,1390,1518,34.938093,-9.856322,37398,0,1,11807,3,11,5,51,1426,109,1,0,6,2244,0,1999,3,1,0,7,4,2,0,6,2,1,1,8,6,0,1,1,2
1,8776,0.0,469,1399,545,34.698766,-2.147466,37194,0,4,15838,9,20,2,103,1576,280,2,0,1,2695,1,2010,3,1,0,11,4,0,2,6,2,2,2,5,3,1,1,1,2
2,34310,25.0,825,686,2048,37.460664,-3.821329,14572,0,5,9074,8,21,4,108,1624,250,1,0,6,2120,1,2009,3,1,0,7,4,4,5,6,2,1,1,0,1,1,2,1,2
3,67743,0.0,1740,263,1852,38.486161,-11.155298,37284,0,7,8982,12,90,63,87,1571,58,1,0,6,2695,1,1986,14,10,5,7,4,0,2,6,2,0,0,3,0,0,2,1,0
4,19728,0.0,20,0,119,31.130847,-1.825359,35528,0,4,7698,4,18,1,26,1687,0,1,0,11,2695,1,0,3,1,0,1,1,0,2,6,2,3,3,5,3,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,436,1210,201,37.169807,-3.253847,513,0,5,5681,6,3,5,16,1090,125,1,0,9,1068,1,1999,3,1,0,9,4,4,5,6,2,1,1,8,6,0,1,1,2
59396,27263,4700.0,177,1212,265,35.249991,-9.070629,24074,0,6,2980,3,11,4,91,353,56,1,0,6,479,1,1996,3,1,0,7,4,2,0,6,2,1,1,6,4,1,1,1,2
59397,37057,0.0,1896,0,2145,34.017087,-8.750434,27926,0,6,8784,10,12,7,59,177,0,1,0,6,2695,0,0,15,11,1,7,4,3,1,1,1,1,1,3,0,0,4,3,2
59398,31282,0.0,884,0,1213,35.861315,-6.378573,29693,0,6,14012,2,1,4,11,1449,0,1,0,6,2695,1,0,8,5,1,7,4,0,2,6,2,2,2,7,5,0,4,3,2


In [11]:
# Validación cruzada
X_train, X_test, y_train, y_test = train_test_split(df3.drop('status_group',axis=1),
                                                    df3.status_group,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify = df3.status_group)

In [31]:
""" model1 = RandomForestClassifier().fit(X_train,y_train)
y_pred = model1.predict(X_test) """

grid_param = {
    'n_estimators': [10, 60, 200],
    'criterion': ['gini', 'entropy'],
    'max_depth': [14, 16, 22, 24],
    'max_features': ['auto', 'sqrt'],
}

In [32]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [33]:
# de https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
RF_Classifier = RandomForestClassifier()

model_grid = GridSearchCV(estimator=RF_Classifier,
                     param_grid=grid_param,
                     scoring='roc_auc',
                     cv=stratified_kfold,
                     n_jobs=-1)
model_grid.fit(X_train, y_train)

print(model_grid.best_score_)
print(model_grid.best_params_)
# No funciona y desconozco el motivo

C:\Users\ACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
120 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
72 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packa

InvalidParameterError: The 'max_features' parameter of RandomForestClassifier must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'sqrt', 'log2'} or None. Got 'auto' instead.

In [29]:
#5.30'
param_grid = {
    'max_features': ['sqrt', 'log2', None],  # O elimina 'auto' si es tu caso
    # Otros parámetros
}

model_grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
model_grid.fit(X_train, y_train)


print(model_grid.best_score_)

print(model_grid.best_params_)

# la prediccion es 0.001 menor que en la V0.0.
# No me queda tiempo para ver el motivo por el que el chunk anterior no ha funcionado

0.8054713804713804
{'max_features': 'log2'}
